<a href="https://colab.research.google.com/github/tosaka-m/japanese_realtime_tts/blob/feature%2Fglowtts_training_script/colab/jsut_model_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Install Requirements
device = 'cpu' # or 'cuda'
assert device in ['cpu', 'cuda']
!apt-get install -y swig mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file


Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
curl is already the newest version (7.58.0-2ubuntu3.9).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils swig
  swig3.0
0 upgraded, 13 newly installed, 0 to remove and 35 

In [2]:
### Install japanese_realtime_tts module
!pip install git+https://github.com/tosaka-m/japanese_realtime_tts.git@feature/glowtts_training_script


  Cloning https://github.com/tosaka-m/japanese_realtime_tts.git (to revision feature/glowtts_training_script) to /tmp/pip-req-build-d0f4kc5x
  Running command git clone -q https://github.com/tosaka-m/japanese_realtime_tts.git /tmp/pip-req-build-d0f4kc5x
  Running command git checkout -b feature/glowtts_training_script --track origin/feature/glowtts_training_script
  Switched to a new branch 'feature/glowtts_training_script'
  Branch 'feature/glowtts_training_script' set up to track remote branch 'feature/glowtts_training_script' from 'origin'.
     |████████████████████████████████| 81kB 5.4MB/s 
     |████████████████████████████████| 2.1MB 21.7MB/s 
     |████████████████████████████████| 276kB 50.8MB/s 
     |████████████████████████████████| 15.9MB 207kB/s 
     |████████████████████████████████| 2.6MB 47.6MB/s 
  Created wheel for japanese-realtime-tts: filename=japanese_realtime_tts-0.0.1-cp36-cp36m-linux_x86_64.whl size=424637 sha256=841c4889496376d2e875d820ce63d6427cb5e2c3cd518

In [8]:
### Download models
!mkdir pretrained_models
!mkdir pretrained_models/glow_tts
!wget -P pretrained_models/glow_tts https://japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com/jsut_v1/glow_tts/epoch_01000.pth
!wget -P pretrained_models/glow_tts https://japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com/jsut_v1/glow_tts/config.yml

!mkdir pretrained_models/mbmelgan
!wget -P pretrained_models/mbmelgan https://japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com/jsut_v1/mbmelgan/epoch_995.pth
!wget -P pretrained_models/mbmelgan https://japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com/jsut_v1/mbmelgan/melgan_config6.yml


mkdir: cannot create directory ‘pretrained_models/mbmelgan’: File exists
--2020-08-23 10:17:19--  https://japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com/jsut_v1/mbmelgan/epoch_01000.pth
Resolving japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com (japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com)... 52.219.4.107
Connecting to japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com (japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com)|52.219.4.107|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2020-08-23 10:17:20 ERROR 403: Forbidden.

--2020-08-23 10:17:23--  https://japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com/jsut_v1/mbmelgan/config.yml
Resolving japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com (japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com)... 52.219.136.3
Connecting to japanese-realtime-tts-models.s3-ap-northeast-1.amazonaws.com (japanese-realtime-tts-models.s3-ap-

In [10]:
### load packages
import yaml
import math
import torch
from torch import nn
import IPython.display as ipd
from jrtts.ParallelWaveGAN.Networks.build_model import build_model as melgan_build_model
from jrtts.ParallelWaveGAN.Utils.losses.pqmf import PQMF
from jrtts.GlowTTS.Networks.build_model import build_model as glowtts_build_model
from jrtts.GlowTTS.AutoEncoder.Networks.build_model import build_model as ae_build_model
from jrtts.text_utils import text2hiragana, hiragana2onso
from jrtts.GlowTTS.Utils.text_utils import TextCleaner
cleaner = TextCleaner()


In [14]:
### Hyperparameters
melgan_config_path = 'pretrained_models/mbmelgan/melgan_config6.yml'
glowtts_config_path = 'pretrained_models/glow_tts/config.yml'

### load configs
melgan_config = yaml.safe_load(open(melgan_config_path))
glowtts_config = yaml.safe_load(open(glowtts_config_path))


### build models and load model parameters
#### MBMelgan
melgan_path = 'pretrained_models/mbmelgan/epoch_995.pth'
melgan_model = melgan_build_model(
    generator_type=melgan_config['generator_type'], 
    discriminator_type=melgan_config['discriminator_type'],
    model_params={
        'generator': melgan_config['generator_params'],
        'discriminator': melgan_config['discriminator_params']
    }
)[0]
melgan_model.load_state_dict(torch.load(melgan_path, map_location='cpu')['model']['generator'])
pqmf = PQMF(4)
melgan_model.eval().to(device)
pqmf.eval().to(device)

#### GlowTTS

glowtts_model_path = 'pretrained_models/glow_tts/epoch_01000.pth'
model = glowtts_build_model(model_params=glowtts_config['model_params'])
model.load_state_dict(torch.load(glowtts_model_path, map_location='cpu')['model'])
_ = model.eval().to(device)


In [15]:
### Def Text 
text = 'こんにちわ、今日はいい天気ですね。'
text = hiragana2onso(text2hiragana(text))
print('roman:%s' % text)
text = cleaner(text)
text = torch.LongTensor(text).unsqueeze(0).to(device)
text_lengths = torch.LongTensor([text.shape[1]]).to(device)


roman: koNniciwa kyouwaiiteNkidesune..


In [19]:
### Text to Speech

noise_scale = 0.4      # std of latent normal distribution 
length_scale = 1.05    # audio length scale 
use_autoenc = True     # whether use autoencoder post processing or not 
use_speakerids = True  

with torch.no_grad():
    (y_gen_tst, *r), attn_gen, *_ = model(
        text, 
        text_lengths, 
        gen=True, 
        noise_scale=noise_scale, 
        length_scale=length_scale)
    
    x = y_gen_tst 
    y = melgan_model(x) #sample[1].transpose(0, 1).unsqueeze(0))
    y = pqmf.synthesis(y) 
    y = y.squeeze(1).squeeze(0).cpu()

In [20]:
sampling_rate = 24000
ipd.Audio(y, rate=sampling_rate)
